In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from src.infra import config

path = '../experiment/test-250605'
opt = config.load_config(path)
opt.path = path
print(opt)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
device_select: auto
network:
  feature_extractor:
    args:
      cache_dir: data/FAUST_r/diffusion
      in_channels: 128
      input_type: wks
      out_channels: 256
    name: DiffusionNet
  fm_solver:
    args:
      
    name: RegularizedFMNet
  name: URSSM
  permutation:
    args:
      tau: 0.07
    name: Similarity
path: ../experiment/test-250605
train:
  checkpoint_interval: 500
  dataset:
    test:
      args:
        data_root: data/FAUST_r/
        num_evecs: 200
        phase: test
        return_corr: True
        return_dist: True
        return_evecs: True
        return_faces: True
      dataloader:
        args:
          batch_size: 1
          shuffle: True
        name: DataLoader
      name: PairFaustDataset
    train:
      args:
        data_root: data/FAUST_r/
        num_evecs: 200
        phas

In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
if 'pth_path' in opt.network:
    print('yes')

## Dataset

In [5]:
import src
from src.infra.registry import DATASET_REGISTRY
DATASET_REGISTRY

{'Dataset': torch.utils.data.dataset.Dataset,
 'SingleFaustDataset': src.dataset.shape_cor.SingleFaustDataset,
 'SingleScapeDataset': src.dataset.shape_cor.SingleScapeDataset,
 'SingleShrec19Dataset': src.dataset.shape_cor.SingleShrec19Dataset,
 'SingleSmalDataset': src.dataset.shape_cor.SingleSmalDataset,
 'SingleDT4DDataset': src.dataset.shape_cor.SingleDT4DDataset,
 'SingleShrec20Dataset': src.dataset.shape_cor.SingleShrec20Dataset,
 'SingleTopKidsDataset': src.dataset.shape_cor.SingleTopKidsDataset,
 'PairDataset': src.dataset.shape_cor.PairDataset,
 'PairFaustDataset': src.dataset.shape_cor.PairFaustDataset,
 'PairScapeDataset': src.dataset.shape_cor.PairScapeDataset,
 'PairShrec19Dataset': src.dataset.shape_cor.PairShrec19Dataset,
 'PairSmalDataset': src.dataset.shape_cor.PairSmalDataset,
 'PairDT4DDataset': src.dataset.shape_cor.PairDT4DDataset,
 'PairShrec20Dataset': src.dataset.shape_cor.PairShrec20Dataset,
 'PairShrec16Dataset': src.dataset.shape_cor.PairShrec16Dataset,
 'Pai

In [6]:
dataset = DATASET_REGISTRY['PairFaustDataset'](
    phase='train',
    data_root='../data/FAUST_r/',
    return_evecs='true',
    return_faces='true',
    num_evecs=200,
    return_corr='false',
    return_dist='false',
)

In [7]:
from src.infra.registry import DATALOADER_REGISTRY
DATALOADER_REGISTRY

{'DataLoader': torch.utils.data.dataloader.DataLoader}

In [8]:
dataloader = DATALOADER_REGISTRY['DataLoader'](
    dataset=dataset,
    batch_size=1,
    shuffle=True,
)
dataloader

In [9]:
from src.utils.tensor import to_device

for batch, data in enumerate(dataloader):
    data = to_device(data, device)
    data_x, data_y = data['first'], data['second']
    print(data_x['name'], data_y['name'])
    break

['tr_reg_030'] ['tr_reg_039']


In [10]:
data

{'first': {'name': ['tr_reg_030'],
  'verts': tensor([[[ 0.0313,  0.5211,  0.1447],
           [ 0.0391,  0.5218,  0.1370],
           [ 0.0239,  0.4825,  0.1326],
           ...,
           [-0.1408,  0.3237,  0.0589],
           [-0.1864, -0.6583, -0.0119],
           [-0.1899, -0.6484, -0.0083]]], device='cuda:0'),
  'faces': tensor([[[  61,    0,   14],
           [  67,    0,  104],
           [ 104,    0,   61],
           ...,
           [4987, 2462, 4988],
           [4988, 3881, 4987],
           [4988, 2462, 2995]]], device='cuda:0'),
  'evecs': tensor([[[ 1.0000, -0.9961, -0.0252,  ...,  1.3570, -3.3923,  5.3566],
           [ 1.0000, -0.9955, -0.0251,  ...,  1.5223, -3.4034,  5.0044],
           [ 1.0000, -0.9791, -0.0246,  ...,  1.1407, -2.3553,  3.3838],
           ...,
           [ 1.0000, -0.8350, -0.0439,  ..., -1.9760, -0.8635,  0.1942],
           [ 1.0000,  1.9174, -2.5915,  ...,  1.3890,  0.8231, -0.4747],
           [ 1.0000,  1.9174, -2.5916,  ...,  1.5511,  0.90

## Model

In [ ]:
from src.infra.registry import MODULE_REGISTRY
MODULE_REGISTRY

{'Similarity': src.model.permutation.Similarity,
 'RegularizedFMNet': src.model.fmap.RegularizedFMNet,
 'URSSM': src.model.urssm.URSSM,
 'DiffusionNet': src.model.diffusionnet.DiffusionNet}

In [ ]:
urssm = MODULE_REGISTRY[opt.network.name](opt).to(device)
urssm

URSSM(
  (feature_extractor): DiffusionNet(
    (first_linear): Linear(in_features=128, out_features=128, bias=True)
    (last_linear): Linear(in_features=128, out_features=256, bias=True)
    (blocks): ModuleList(
      (0): DiffusionNetBlock(
        (diffusion): LearnedTimeDiffusion()
        (gradient_features): SpatialGradientFeatures(
          (A_re): Linear(in_features=128, out_features=128, bias=False)
          (A_im): Linear(in_features=128, out_features=128, bias=False)
        )
        (mlp): MiniMLP(
          (miniMLP_linear_000): Linear(in_features=384, out_features=128, bias=True)
          (miniMLP_activation_000): ReLU()
          (miniMLP_dropout_001): Dropout(p=0.5, inplace=False)
          (miniMLP_linear_001): Linear(in_features=128, out_features=128, bias=True)
          (miniMLP_activation_001): ReLU()
          (miniMLP_dropout_002): Dropout(p=0.5, inplace=False)
          (miniMLP_linear_002): Linear(in_features=128, out_features=128, bias=True)
        )
  

In [13]:
network_path = '../checkpoints/faust.pth'
urssm.feature_extractor.load_state_dict(
    torch.load(network_path)['networks']['feature_extractor']
)
print(f'Loaded pretrain weights from {network_path}')

Loaded pretrain weights from ../checkpoints/faust.pth


In [14]:
infer = urssm(data)
infer

{'Cxy': tensor([[[ 9.6219e-01,  1.5338e-02, -2.4659e-02,  ..., -6.4113e-05,
            3.6756e-04, -2.9156e-04],
          [-1.8863e-02,  9.6023e-01, -2.4035e-02,  ...,  6.9799e-04,
            6.4222e-04, -3.5222e-04],
          [ 2.1723e-02,  2.0744e-02, -1.0053e+00,  ...,  4.7211e-04,
           -3.9685e-05,  3.0762e-04],
          ...,
          [ 3.3094e-04, -2.4268e-04,  8.4136e-05,  ...,  2.1029e-01,
           -5.6321e-01,  4.6303e-01],
          [-1.8278e-04,  7.3649e-04, -5.9551e-04,  ...,  4.3372e-02,
           -5.3321e-01, -2.8000e-01],
          [ 2.1597e-04,  2.9710e-04, -1.7957e-03,  ..., -1.3474e-02,
            1.0295e-01, -8.0878e-02]]], device='cuda:0', grad_fn=<CatBackward0>),
 'Cyx': tensor([[[ 9.9082e-01, -1.7312e-02, -1.6508e-02,  ...,  7.7828e-04,
           -1.1171e-04,  4.6791e-05],
          [ 1.7325e-02,  1.0037e+00, -1.9745e-02,  ...,  6.1401e-04,
            6.0568e-04, -5.8254e-04],
          [ 2.1323e-02,  2.3070e-02, -9.8167e-01,  ...,  4.0590e-04,
  

## Loss

In [15]:
from src.infra.registry import LOSS_REGISTRY
LOSS_REGISTRY

{'MSELoss': torch.nn.modules.loss.MSELoss,
 'SquaredFrobeniusLoss': src.loss.fmap.SquaredFrobeniusLoss,
 'SURFMNetLoss': src.loss.fmap.SURFMNetLoss,
 'SURFMNetLoss_wrap': src.loss.fmap.SURFMNetLoss_wrap,
 'SpatialSpectralAlignmentLoss': src.loss.fmap.SpatialSpectralAlignmentLoss,
 'SpatialSpectralAlignmentLoss_wrap': src.loss.fmap.SpatialSpectralAlignmentLoss_wrap,
 'PartialFmapsLoss': src.loss.fmap.PartialFmapsLoss,
 'DirichletLoss': src.loss.dirichlet.DirichletLoss,
 'CompositeLoss': src.loss.composite.CompositeLoss}

In [16]:
loss_dict = {}

for name, loss in opt.train.loss.items():
    loss_dict[name] = {
        'fn': LOSS_REGISTRY[loss['name']](**loss['args']).to(device),
        'weight': loss['weight'],
    }

loss_dict

{'surfm_loss': {'fn': SURFMNetLoss_wrap(
    (squared_frobenius): SquaredFrobeniusLoss()
  ),
  'weight': 1.0},
 'align_loss': {'fn': SpatialSpectralAlignmentLoss_wrap(
    (squared_frobenius): SquaredFrobeniusLoss()
  ),
  'weight': 1.0}}

In [17]:
for name, loss in loss_dict.items():
    print(name, loss['fn'](infer, data))

surfm_loss tensor(57.3256, device='cuda:0', grad_fn=<AddBackward0>)
align_loss tensor(49.5927, device='cuda:0', grad_fn=<AddBackward0>)


## Metrics

In [18]:
from src.infra.registry import METRIC_REGISTRY
METRIC_REGISTRY

{'L1Loss': torch.nn.modules.loss.L1Loss,
 'MSELoss': torch.nn.modules.loss.MSELoss,
 'MeanDiffRatio': src.metric.stats.MeanDiffRatio,
 'StdDiffRatio': src.metric.stats.StdDiffRatio,
 'calculate_geodesic_error': <function src.metric.geodist.calculate_geodesic_error(dist_x, corr_x, corr_y, p2p, return_mean=True)>,
 'GeodesicDist': src.metric.geodist.GeodesicDist,
 'plot_pck': <function src.metric.geodist.plot_pck(geo_err, threshold=0.1, steps=40)>}

In [19]:
metric_dict = {}

for name, metric in opt.train.metric.items():
    metric_dict[name] = METRIC_REGISTRY[metric['name']](**metric['args']).to(device)

metric_dict

{'geodist': GeodesicDist()}

In [20]:
for name, metric in metric_dict.items():
    print(name, metric(infer, data))

geodist 0.017233284190297127


## Training loop

In [21]:
from src.infra.registry import OPTIMIZER_REGISTRY
OPTIMIZER_REGISTRY

{'SGD': torch.optim.sgd.SGD,
 'Adam': torch.optim.adam.Adam,
 'AdamW': torch.optim.adamw.AdamW,
 'RMSprop': torch.optim.rmsprop.RMSprop,
 'Adagrad': torch.optim.adagrad.Adagrad,
 'Adadelta': torch.optim.adadelta.Adadelta}

In [22]:
optimizer = OPTIMIZER_REGISTRY[opt.train.optimizer.name](
    params=urssm.parameters(),
    **opt.train.optimizer.args,
)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [23]:
urssm.train()

for batch, data in enumerate(dataloader):
    data = to_device(data, device)
    infer = urssm(data)
    loss_val = 0
    
    for name, loss in loss_dict.items():
        loss_value = loss['fn'](infer, data) * loss['weight']
        loss_val += loss_value
    
    print(f'batch {batch} loss: {loss_val.item()}')

    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    if batch > 10:
        break

batch 0 loss: 156.3905029296875
batch 1 loss: 161.17811584472656
batch 2 loss: 131.73617553710938
batch 3 loss: 188.54330444335938
batch 4 loss: 147.65383911132812
batch 5 loss: 200.42495727539062
batch 6 loss: 154.72117614746094
batch 7 loss: 154.2474822998047
batch 8 loss: 135.6914520263672
batch 9 loss: 149.32748413085938
batch 10 loss: 158.50827026367188
batch 11 loss: 180.33425903320312


## Inference mode

In [24]:
infer = urssm.inference(data)
infer

{'Cxy': tensor([[[-9.6680e-01, -7.6220e-03, -5.4017e-03,  ..., -3.8725e-04,
            5.4300e-04, -1.3607e-04],
          [-8.4055e-03,  9.5202e-01, -9.6099e-02,  ..., -3.6205e-04,
            2.4001e-05, -6.0918e-04],
          [ 4.7266e-03,  1.5281e-01,  9.4750e-01,  ..., -9.4869e-04,
           -2.7597e-04, -2.7404e-04],
          ...,
          [-1.7067e-03, -5.2618e-05,  1.9227e-04,  ..., -8.9134e-02,
            1.8322e-01,  1.3075e-01],
          [-9.6023e-04, -8.8564e-04,  3.0949e-03,  ...,  3.2237e-03,
            3.0350e-01,  2.2264e-01],
          [-2.4657e-04,  1.4766e-03, -3.0290e-04,  ..., -2.8351e-02,
           -1.3229e-01,  7.4403e-02]]], device='cuda:0', grad_fn=<CatBackward0>),
 'Pyx': [tensor([[ 2.0774e-02,  3.1640e-02,  2.9294e-03,  ..., -4.9916e-05,
            4.7147e-05,  4.9359e-05],
          [ 2.2859e-02,  3.2898e-02,  5.4672e-03,  ..., -5.1118e-05,
            1.1208e-05,  2.3583e-05],
          [ 2.3053e-02,  3.3987e-02,  6.0531e-03,  ..., -5.9679e-05,
  